# CIFAR-10 Challange - 김영인

1. wide-resnet (28*10)
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize(mean, std)
4. \+ regularization(weight decay, dropout)
5. \+ Optimizer(momentum, scheduler)

In [1]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [11]:
learning_rate = 0.1
scheduler_step = 60
scheduler_gamma = 0.2
training_epochs = 200
batch_size = 64
momentum = 0.9
weight_decay = 5e-4
depth = 28
widen_factor = 10
dropout_rate = 0.3

## Load & Preprocess Data

In [12]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(32), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [13]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

class wide_basic(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1):
        super(wide_basic, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)

        return out

class Wide_ResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(Wide_ResNet, self).__init__()
        self.in_planes = 16

        assert ((depth-4)%6 ==0), 'Wide-resnet depth should be 6n+4'
        n = (depth-4)/6
        k = widen_factor

        print('| Wide-Resnet %dx%d' %(depth, k))
        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3,nStages[0])
        self.layer1 = self._wide_layer(wide_basic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(wide_basic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(wide_basic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=0.9)
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(int(num_blocks)-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out

## Load Model

In [14]:
net=Wide_ResNet(depth, widen_factor, dropout_rate, 10)
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

| Wide-Resnet 28x10


## Train

In [15]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    print(f'=== # {epoch} epoch start! ===')
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        
print('Finished Training')

=== # 0 epoch start! ===
[1,   100] loss: 2.229
[1,   200] loss: 2.010
[1,   300] loss: 1.911
[1,   400] loss: 1.863
[1,   500] loss: 1.827
[1,   600] loss: 1.775
[1,   700] loss: 1.768
=== # 1 epoch start! ===
[2,   100] loss: 1.689
[2,   200] loss: 1.631
[2,   300] loss: 1.620
[2,   400] loss: 1.572
[2,   500] loss: 1.529
[2,   600] loss: 1.522
[2,   700] loss: 1.497
=== # 2 epoch start! ===
[3,   100] loss: 1.443
[3,   200] loss: 1.420
[3,   300] loss: 1.408
[3,   400] loss: 1.401
[3,   500] loss: 1.379
[3,   600] loss: 1.367
[3,   700] loss: 1.333
=== # 3 epoch start! ===
[4,   100] loss: 1.357
[4,   200] loss: 1.322
[4,   300] loss: 1.322
[4,   400] loss: 1.299
[4,   500] loss: 1.281
[4,   600] loss: 1.293
[4,   700] loss: 1.295
=== # 4 epoch start! ===
[5,   100] loss: 1.229
[5,   200] loss: 1.262
[5,   300] loss: 1.284
[5,   400] loss: 1.231
[5,   500] loss: 1.250
[5,   600] loss: 1.240
[5,   700] loss: 1.228
=== # 5 epoch start! ===
[6,   100] loss: 1.205
[6,   200] loss: 1.179

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


[8,   600] loss: 1.111
[8,   700] loss: 1.106
=== # 8 epoch start! ===


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


[9,   100] loss: 1.120
[9,   200] loss: 1.137
[9,   300] loss: 1.107
[9,   400] loss: 1.105
[9,   500] loss: 1.088
[9,   600] loss: 1.071
[9,   700] loss: 1.105
=== # 9 epoch start! ===


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers

Traceback (most recent call last):
self._shutdown_workers()      File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
      File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'

AssertionError:   File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site

[58,   100] loss: 0.949
[58,   200] loss: 0.940
[58,   300] loss: 0.984
[58,   400] loss: 0.962
[58,   500] loss: 0.985
[58,   600] loss: 0.975
[58,   700] loss: 0.960
=== # 58 epoch start! ===


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 138, in join
    Exception ignored in: assert self._parent_pid == os.getpid(), 'can only join a child process'
<function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>AssertionError: 
Traceback (most recent call last):
can only join a child process  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__

    self._shutdown_workers()
  File "/home/pirl/anaconda3/e

[59,   100] loss: 0.985


KeyboardInterrupt: 

## Test

In [16]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4a246a5f0>
Traceback (most recent call last):
  File "/home/pirl/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/pirl/anaconda3/e

Accuracy of plane : 66.400000 %
Accuracy of   car : 77.200000 %
Accuracy of  bird : 48.700000 %
Accuracy of   cat : 42.500000 %
Accuracy of  deer : 66.900000 %
Accuracy of   dog : 56.500000 %
Accuracy of  frog : 69.300000 %
Accuracy of horse : 76.100000 %
Accuracy of  ship : 77.000000 %
Accuracy of truck : 72.500000 %
Mean Accuracy :  65.31
